In [42]:
import pandas as pd
import numpy as np
import os 
import pickle
import nltk
from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer  
lemmatizer = WordNetLemmatizer() 
english_stopwords = set(stopwords.words('english')) 
from sklearn.naive_bayes import GaussianNB
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
import re
import string
from re import search

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/utsavbaghela/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/utsavbaghela/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/utsavbaghela/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [43]:
#Dataset access and path setting
dir="20_newsgroups"
list_of_files = os.listdir(dir)

files_path=[]
for f in list_of_files:
    files_path.append(dir+'/'+f)

In [44]:
del(files_path[0])

In [45]:
del(files_path[4])

In [46]:
files_path

['20_newsgroups/rec.sport.hockey',
 '20_newsgroups/sci.med',
 '20_newsgroups/talk.politics.misc',
 '20_newsgroups/comp.graphics',
 '20_newsgroups/sci.space']

In [47]:
files_label_mapping = {
    'comp.graphics': 3,
    'sci.med': 1,
    'talk.politics.misc': 2,
    'rec.sport.hockey': 0,
    'sci.space': 4
}


In [48]:
files_path

['20_newsgroups/rec.sport.hockey',
 '20_newsgroups/sci.med',
 '20_newsgroups/talk.politics.misc',
 '20_newsgroups/comp.graphics',
 '20_newsgroups/sci.space']

In [49]:
all_files_path=[]
labels=[]
j=0
for f in files_path:
    list_of_files = os.listdir(f)
    for i in list_of_files:
        all_files_path.append(f+'/'+i)
        labels.append(j);
    j=j+1;
        

In [50]:
labels[3000]

3

### 1. Data Preprocessing

In [51]:
from nltk.corpus import stopwords

lemma = WordNetLemmatizer() 
all_stopWords = set(stopwords.words('english'))


#data pre-processing
def pre_processing(content):
    content = re.sub(r'[^a-zA-Z0-9]+', ' ', content)
    content = content.lower()
    content = [word for word in content.split() if word not in all_stopWords]
    content = [word for word in content if len(word) > 2]
    content = [lemmatizer.lemmatize(word) for word in content]
    return " ".join(content).strip()
    

In [52]:
datalist=[]
for f in all_files_path:
    with open(f, 'r',errors='ignore',encoding='latin-1') as file:              
            content = file.read().replace('\n', '')
            datalist.append(pre_processing(content))

In [53]:
datalist=np.array(datalist)


In [54]:
labels=np.array(labels)

## 2. Split your dataset randomly into train: test ratio.

In [55]:
def split_train_and_test_randomly(ratio_of_split):
    len_datalist = len(datalist)
    len_of_train_data = int(len_datalist * ratio_of_split)
    
    random_indices = np.random.choice(len_datalist, size=len_of_train_data)
    
    data_train = datalist[random_indices]
    data_train_lables = labels[random_indices]
    
    data_test = datalist[np.setdiff1d(range(len_datalist), random_indices)]
    data_test_lables = labels[np.setdiff1d(range(len_datalist), random_indices)]
    
    return (data_train, data_train_lables, data_test, data_test_lables)

In [56]:
data_train, data_train_lables, data_test, data_test_lables = split_train_and_test_randomly(0.8)
print("Train Size: ", len(data_train), len(data_train_lables))
print("Test Size: ", len(data_test), len(data_test_lables))

Train Size:  4000 4000
Test Size:  2275 2275


In [57]:
len(data_train)

4000

### 3.Implement the TF-ICF scoring technique for efficient feature selection.

In [58]:
# Calculating Class Frequency
Class_frequency = {}
for index,i in enumerate(data_train):
    for word in i.split(" "):
        class_of_word=labels[index]
        if word not in Class_frequency:
            Class_frequency[word] = set()
        Class_frequency[word].add(class_of_word)

In [59]:
Class_frequency

{'path': {0, 1, 2, 3},
 'cantaloupe': {0, 1, 2, 3},
 'srv': {0, 1, 2, 3},
 'cmu': {0, 1, 2, 3},
 'edu': {0, 1, 2, 3},
 'magnesium': {0, 1, 2, 3},
 'club': {0, 1, 2, 3},
 'news': {0, 1, 2, 3},
 'sei': {0, 1, 2, 3},
 'ci': {0, 1, 2, 3},
 'ohio': {0, 1, 2, 3},
 'state': {0, 1, 2, 3},
 'zaphod': {0, 1, 2, 3},
 'mp': {0, 1, 2, 3},
 'usc': {0, 1, 2, 3},
 'rutgers': {0, 1, 2, 3},
 'uwvax': {0, 3},
 'yar': {0, 1, 3},
 'wisc': {0, 1, 2, 3},
 'oehlerfrom': {0, 3},
 'oehler': {0, 1, 3},
 'wonko': {0, 1, 3},
 'sane': {0, 1, 2, 3},
 'newsgroups': {0, 1, 2, 3},
 'comp': {0, 1, 2, 3},
 'graphicssubject': {0, 1, 2, 3},
 'bit': {0, 1, 2, 3},
 'graphic': {0, 1, 2, 3},
 'keywords': {0, 1, 2, 3},
 'alpha': {0, 1, 2, 3},
 'channel': {0, 1, 2, 3},
 'imagemessage': {0, 1, 2, 3},
 '1993apr24': {0, 1, 2, 3},
 '201117': {0, 1, 3},
 '26232': {0, 1, 3},
 'date': {0, 1, 2, 3},
 'apr': {0, 1, 2, 3},
 'gmtsender': {0, 1, 2, 3},
 'organization': {0, 1, 2, 3},
 'university': {0, 1, 2, 3},
 'wisconsin': {0, 1, 2, 3},
 

In [60]:
#Calculating Term Frequency
Term_frequency = {0: {},1: {},2: {},3: {},4: {}}
for index,i in enumerate(data_train):
    for word in i.split(" "):
        dtl_index=data_train_lables[index]
        if word not in Term_frequency[dtl_index]:
            Term_frequency[dtl_index][word] = 0
        Term_frequency[dtl_index][word] += 1

In [61]:
Term_frequency

{0: {'path': 352,
  'cantaloupe': 790,
  'srv': 984,
  'cmu': 1970,
  'edu': 5669,
  'da': 245,
  'news': 1412,
  'harvard': 304,
  'ogicse': 27,
  'uwm': 102,
  'rpi': 47,
  'columbia': 256,
  'cunixb': 90,
  'gldfrom': 20,
  'gld': 111,
  'gary': 120,
  'dare': 72,
  'newsgroups': 723,
  'rec': 885,
  'sport': 962,
  'hockeysubject': 344,
  'many': 177,
  'european': 67,
  'nhlmessage': 11,
  '1993apr6': 44,
  '204743': 8,
  '21314': 4,
  'date': 557,
  'apr': 811,
  'gmtarticle': 27,
  '21314references': 4,
  'rauser': 23,
  '734062608': 13,
  'sfu': 80,
  'sender': 313,
  'usenet': 249,
  'network': 82,
  'reply': 79,
  'organization': 277,
  'phd': 20,
  'halllines': 4,
  '17nntp': 6,
  'posting': 406,
  'host': 379,
  'eduyou': 4,
  'right': 213,
  'sick': 16,
  'seeing': 19,
  'white': 18,
  'guy': 143,
  'skatesmyself': 4,
  'vancouver': 206,
  'canuck': 94,
  'half': 17,
  'woman': 55,
  'overall': 14,
  'one': 439,
  'third': 130,
  'oriental': 4,
  'gladly': 4,
  'volunteer'

In [62]:
import math
def Cal_ICF(j):
    return math.log(5/len(Class_frequency[j]))

In [63]:
#ICF Values Calculation
ICF_values = {}
for j in Class_frequency.keys():
    ICF_values[j] = Cal_ICF(j)

In [64]:
ICF_values

{'path': 0.22314355131420976,
 'cantaloupe': 0.22314355131420976,
 'srv': 0.22314355131420976,
 'cmu': 0.22314355131420976,
 'edu': 0.22314355131420976,
 'magnesium': 0.22314355131420976,
 'club': 0.22314355131420976,
 'news': 0.22314355131420976,
 'sei': 0.22314355131420976,
 'ci': 0.22314355131420976,
 'ohio': 0.22314355131420976,
 'state': 0.22314355131420976,
 'zaphod': 0.22314355131420976,
 'mp': 0.22314355131420976,
 'usc': 0.22314355131420976,
 'rutgers': 0.22314355131420976,
 'uwvax': 0.9162907318741551,
 'yar': 0.5108256237659907,
 'wisc': 0.22314355131420976,
 'oehlerfrom': 0.9162907318741551,
 'oehler': 0.5108256237659907,
 'wonko': 0.5108256237659907,
 'sane': 0.22314355131420976,
 'newsgroups': 0.22314355131420976,
 'comp': 0.22314355131420976,
 'graphicssubject': 0.22314355131420976,
 'bit': 0.22314355131420976,
 'graphic': 0.22314355131420976,
 'keywords': 0.22314355131420976,
 'alpha': 0.22314355131420976,
 'channel': 0.22314355131420976,
 'imagemessage': 0.223143551314

In [65]:
#TF_ICF Values Calculation
TF_ICF_values = {}
for k in range(0,5):
    TF_ICF_values[k] = {}
    for w in Term_frequency[k]:
        TF_ICF_values[k][w] =ICF_values[w]*Term_frequency[k][w]

In [66]:
TF_ICF_values

{0: {'path': 78.54653006260183,
  'cantaloupe': 176.2834055382257,
  'srv': 219.57325449318242,
  'cmu': 439.5927960889932,
  'edu': 1265.0007924002552,
  'da': 54.670170071981396,
  'news': 315.07869445566416,
  'harvard': 67.83563959951977,
  'ogicse': 6.024875885483664,
  'uwm': 22.760642234049396,
  'rpi': 10.487746911767859,
  'columbia': 57.1247491364377,
  'cunixb': 20.08291961827888,
  'gldfrom': 4.462871026284195,
  'gld': 24.768934195877282,
  'gary': 26.777226157705172,
  'dare': 16.066335694623103,
  'newsgroups': 161.33278760017367,
  'rec': 197.48204291307565,
  'sport': 214.6640963642698,
  'hockeysubject': 76.76138165208816,
  'many': 39.49640858261513,
  'european': 14.950617938052055,
  'nhlmessage': 2.4545790644563072,
  '1993apr6': 9.818316257825229,
  '204743': 7.330325854993241,
  '21314': 3.6651629274966204,
  'date': 124.29095808201484,
  'apr': 180.9694201158241,
  'gmtarticle': 6.024875885483664,
  '21314references': 3.6651629274966204,
  'rauser': 5.132301680

In [67]:
# Value of k
defined_value_of_k = 200

In [68]:

effective_vocabulary = set()
for i in range(5):
    non_list=dict(sorted(TF_ICF_values[i].items(), key=lambda y: y[1], reverse=True)).keys()
    b = list(non_list)
    effective_vocabulary = effective_vocabulary.union(set(b[:defined_value_of_k]))


In [69]:
len(effective_vocabulary)

562

In [70]:
def generate_data(data,effective_vocabulary):
    content_new = []
    for t in data:
        line = []
        for word in t.split(" "):
            if word in effective_vocabulary:
                line.append(word)
        content_new.append(" ".join(line).strip())

    return content_new

In [71]:
from sklearn.feature_extraction.text import TfidfVectorizer

train_data_new = generate_data(data_train,effective_vocabulary)
test_data_new = generate_data(data_test,effective_vocabulary)

In [72]:
from sklearn.feature_extraction.text import TfidfVectorizer
def create_features_top_k(training, testing):
    vect = TfidfVectorizer()
    train = vect.fit_transform(training)
    test = vect.transform(testing)
    return (train, test)

In [73]:
train,test = create_features_top_k(train_data_new, test_data_new)

In [74]:
train.toarray()

array([[0.        , 0.        , 0.        , ..., 0.        , 0.05124299,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.01512805,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.04811951,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.02893773,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [75]:
test.toarray()

array([[0.05217955, 0.        , 0.        , ..., 0.        , 0.02294959,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.03527973,
        0.        ]])

### Implemeting Naive Bias

In [76]:
# reference : https://stackoverflow.com/questions/60969884/multinomial-naive-bayes-for-python-from-scratch

class naive_bais_class:
    def __init__(self, alpha=1):
        self.alpha = alpha 

    def fit(self, X_train, y_train):
        m, n = X_train.shape
        self._classes = np.unique(y_train)
        n_classes = len(self._classes)

        # init: Prior & Likelihood
        self._priors = np.zeros(n_classes)
        self._likelihoods = np.zeros((n_classes, n))

        # Get Prior and Likelihood
        for idx, c in enumerate(self._classes):
            X_train_c = X_train[c == y_train]
            self._priors[idx] = X_train_c.shape[0] / m 
            self._likelihoods[idx, :] = ((X_train_c.sum(axis=0)) + self.alpha) / (np.sum(X_train_c.sum(axis=0) + self.alpha))


    def predict(self, X_test):
        return [self._predict(x_test) for x_test in X_test]

    def _predict(self, x_test):
        # Calculate posterior for each class
        posteriors = []
        for idx, c in enumerate(self._classes):
            prior_c = np.log(self._priors[idx])
            likelihoods_c = self.calc_likelihood(self._likelihoods[idx,:], x_test)
            posteriors_c = np.sum(likelihoods_c) + prior_c
            posteriors.append(posteriors_c)

        return self._classes[np.argmax(posteriors)]

    def calc_likelihood(self, cls_likeli, x_test):
        return np.log(cls_likeli) * x_test

    def score(self, X_test, y_test):
        y_pred = self.predict(X_test)
        return np.sum(y_pred == y_test)/len(y_test)

In [77]:
modelNB2 = naive_bais_class()

modelNB2.fit(train.toarray(),data_train_lables)

predict_train = modelNB2.predict(train.toarray())
predict_test = modelNB2.predict(test.toarray())



In [78]:
def compute_accuracy(y_true, y_pred):
    correct_predictions = 0
    # iterate over each label and check
    for true, predicted in zip(y_true, y_pred):
        if true == predicted:
            correct_predictions += 1
    # compute the accuracy
    accuracy = correct_predictions/len(y_true)
    return accuracy

### Test your classifier on testing data and report the confusion matrix and overall accuracy.

In [79]:
#Accuracy: split: 80:20

print("Training Accuracy:")
print(compute_accuracy(data_train_lables,predict_train))
print("Testing Accuracy:")
print(compute_accuracy(data_test_lables,predict_test))




Training Accuracy:
0.983
Testing Accuracy:
0.9723076923076923


In [80]:
#Confution Matrix:

from sklearn.metrics import confusion_matrix
confusion_matrix(data_test_lables,predict_test)


array([[460,   0,   0,   2,   1],
       [  3, 410,   2,  12,   8],
       [  1,   1, 451,   2,   4],
       [  1,   2,   0, 441,   3],
       [  0,   8,   3,  10, 450]])

### Split on 50:50, 70:30, and 80:20 

In [81]:
#split 50:50

data_train2, data_train_lables2, data_test2, data_test_lables2 = split_train_and_test_randomly(0.5)

train_data_new2 = generate_data(data_train2,effective_vocabulary)
test_data_new2 = generate_data(data_test2,effective_vocabulary)

train2, test2 = create_features_top_k(train_data_new2, test_data_new2)

modelNB2 = naive_bais_class()

modelNB2.fit(train2.toarray(),data_train_lables2)

predict_train2 = modelNB2.predict(train2.toarray())
predict_test2 = modelNB2.predict(test2.toarray())


#Accuracy:

print("Training Accuracy:")
print(compute_accuracy(data_train_lables2,predict_train2))
print("Testing Accuracy:")
print(compute_accuracy(data_test_lables2,predict_test2))

#Confution Matrix:

from sklearn.metrics import confusion_matrix
confusion_matrix(data_test_lables2,predict_test2)

Training Accuracy:
0.9776
Testing Accuracy:
0.9737101544528426


array([[588,   0,   0,   2,   0],
       [  2, 582,   2,  21,   6],
       [  3,   1, 603,   2,   2],
       [  0,   2,   0, 592,   5],
       [  1,  10,   7,  14, 598]])

In [82]:
#split 70:30

data_train3, data_train_lables3, data_test3, data_test_lables3 = split_train_and_test_randomly(0.7)

train_data_new3 = generate_data(data_train3,effective_vocabulary)
test_data_new3 = generate_data(data_test3,effective_vocabulary)

train3, test3 = create_features_top_k(train_data_new3, test_data_new3)

modelNB2 = naive_bais_class()

modelNB2.fit(train3.toarray(),data_train_lables3)

predict_train3 = modelNB2.predict(train3.toarray())
predict_test3 = modelNB2.predict(test3.toarray())


#Accuracy:

print("Training Accuracy:")
print(compute_accuracy(data_train_lables3,predict_train3))
print("Testing Accuracy:")
print(compute_accuracy(data_test_lables3,predict_test3))

#Confution Matrix:

from sklearn.metrics import confusion_matrix
confusion_matrix(data_test_lables3,predict_test3)

Training Accuracy:
0.9854285714285714
Testing Accuracy:
0.9717400080742834


array([[470,   0,   0,   3,   1],
       [  1, 449,   2,  11,  12],
       [  1,   3, 492,   1,   5],
       [  2,   0,   0, 516,   5],
       [  2,   8,   3,  10, 480]])